In [20]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.transforms import SentencePieceTokenizer
from torchtext.models import RobertaClassificationHead
from torchtext.models import XLMR_BASE_ENCODER

from torchtext.vocab import GloVe
import numpy as np
from torch.nn.functional import pad


In [11]:
test = "Ahoj Michal . ! Super oo avc"

In [12]:
tokenizer = get_tokenizer("basic_english")
tokenizer(test)

['ahoj', 'michal', '.', '!', 'super', 'oo', 'avc']

In [13]:
glv = GloVe(name="6B", dim=50)

In [14]:
tens = glv.get_vecs_by_tokens(tokenizer(test))

In [15]:
import pandas as pd
data = pd.read_csv("../../../data-davidson/data/labeled_data.csv")
data = data.drop(["Unnamed: 0", "count", "hate_speech", "offensive_language", "neither"], axis=1)
data = data.reindex(columns=["tweet", "class"])
data = data.to_numpy()

In [184]:
MAX_LENGTH = 50
DIMS = 50

In [200]:
def collate(batch):
    x_arr, y_arr = [], []
    for tweet, label in batch:
        tweet = glv.get_vecs_by_tokens(tokenizer(tweet))[:MAX_LENGTH, :]
        padding = torch.zeros(MAX_LENGTH-tweet.shape[0], DIMS)
        tweet = torch.cat((tweet, padding), dim=0)
        x_arr.append(tweet.unsqueeze(0))
        y_arr.append(label)

    x = torch.cat(x_arr)
    y = torch.tensor(y_arr)

    return x, y
    # print(len(batch[0]))

In [180]:
torch.cat([torch.tensor([[0.0,1.0,0]]), torch.tensor([[0.0,1.0,0]])])

tensor([[0., 1., 0.],
        [0., 1., 0.]])

In [201]:
from torch.utils.data import DataLoader

dl = DataLoader(data, batch_size=16, collate_fn=collate)

In [34]:
data = pd.read_csv("../../../data-davidson/data/labeled_data.csv")
data = data.drop(["Unnamed: 0", "count", "hate_speech", "offensive_language", "neither"], axis=1)
data = data.reindex(columns=["tweet", "class"])
data.to_numpy().shape

(24783, 2)

In [40]:
np.unique(data.to_numpy()[:, 1], return_counts=True)


(array([0, 1, 2], dtype=object), array([ 1430, 19190,  4163]))

In [25]:
print(np.unique(data[:, 1], return_counts=True)[1][0] / data.shape[0])
print(np.unique(data[:, 1], return_counts=True)[1][1] / data.shape[0])
print(np.unique(data[:, 1], return_counts=True)[1][2] / data.shape[0])


0.05770084332001776
0.7743211072105879
0.16797804946939435


In [44]:
data = pd.read_csv("../../../data-davidson/data/labeled_data.csv")
data = data.drop(["Unnamed: 0", "count", "hate_speech", "offensive_language", "neither"], axis=1)
data = data.reindex(columns=["tweet", "class"])

In [49]:
df = data.drop(
    data[data["class"] == 1]
    .sample(frac=0.92)
    .index
)

df = df.drop(
    df[df["class"] == 2]
    .sample(frac=0.65)
    .index
).to_numpy()

print(np.unique(df[:, 1], return_counts=True)[1][0] / df.shape[0])
print(np.unique(df[:, 1], return_counts=True)[1][1] / df.shape[0])
print(np.unique(df[:, 1], return_counts=True)[1][2] / df.shape[0])

0.32338308457711445
0.3471279963817277
0.32948891904115785
